# Deploy Global Economy Data as an API

This notebook demonstrates how to expose your processed dataset using **FastAPI** so others can query data programmatically.

**Endpoints we'll build:**
- `/` - Welcome message + basic info
- `/summary` - Summary statistics
- `/region/{region}` - Get all data for a specific region
- `/country/{code}` - Lookup by country code

## 1️⃣ Install Dependencies

In [ ]:
%pip install fastapi uvicorn pandas nest_asyncio pyngrok --quiet

## 2️⃣ Load Processed Dataset

In [ ]:
import pandas as pd

# Load the processed dataset
DATA_FILE = "D:\courses\Data Science\Projects\Python\world-economy-bi-project\data\processed_data.csv"
df = pd.read_csv(DATA_FILE)
print(f"✅ Data loaded with {df.shape[0]} rows and {df.shape[1]} columns.")
df.head()

## 3️⃣ Build FastAPI App

In [ ]:
from fastapi import FastAPI, HTTPException
from fastapi.responses import JSONResponse
import nest_asyncio
import uvicorn

app = FastAPI(title="Global Economy API", description="Query World Bank + HDI data", version="1.0")

@app.get("/")
async def root():
    return {"message": "🌍 Welcome to the Global Economy API!", "rows": len(df), "columns": list(df.columns)}

@app.get("/summary")
async def summary():
    return JSONResponse(content=df.describe(include="all").to_dict())

@app.get("/region/{region}")
async def get_region(region: str):
    region_df = df[df["Region"].str.lower() == region.lower()]
    if region_df.empty:
        raise HTTPException(status_code=404, detail=f"Region '{region}' not found")
    return JSONResponse(content=region_df.to_dict(orient="records"))

@app.get("/country/{code}")
async def get_country(code: str):
    country_df = df[df["Country Code"].str.upper() == code.upper()]
    if country_df.empty:
        raise HTTPException(status_code=404, detail=f"Country Code '{code}' not found")
    return JSONResponse(content=country_df.to_dict(orient="records"))


## 4️⃣ Run Locally in Notebook

In [ ]:
# Allow uvicorn to run inside Jupyter
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)

## 5️⃣ Test Your API

Once the server is running, open:
- [http://127.0.0.1:8000/](http://127.0.0.1:8000/) → Welcome page
- [http://127.0.0.1:8000/docs](http://127.0.0.1:8000/docs) → Interactive API Docs

Example calls:

In [ ]:
# import requests
# res = requests.get("http://127.0.0.1:8000/region/Asia")
# res.json()[:3]  # first 3 records

## 6️⃣ Deployment Options

- **Render.com**: Simple free tier for API hosting
- **Railway.app**: Easy CI/CD + auto-deploy from GitHub
- **Hugging Face Spaces (Gradio/Streamlit)**: Free data app hosting

### Recommended:
- Create `main.py` (same as app code above)
- Add `requirements.txt` with:
  ```
  fastapi
  uvicorn
  pandas
  ```